In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chinmaychahar","key":"a26d902805dabf46eb9b538e42790666"}'}

In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets list

ref                                                                title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
themrityunjaypathak/covid-cases-and-deaths-worldwide               Covid Cases and Deaths WorldWide                  8KB  2023-02-01 12:22:51           6209        183  1.0              
amaanansari09/top-100-songs                                        Top 100 songs                                     6KB  2023-02-16 18:55:35            877         32  1.0              
datascientistanna/customers-dataset                                Shop Customer Data                               23KB  2023-02-07 18:42:21           4779        120  1.0              
thedevastator/airbnb-prices-in-european-cities                   

In [11]:
!kaggle datasets list -s IIIT 

ref                                                         title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
tanlikesmath/the-oxfordiiit-pet-dataset                     The Oxford-IIIT Pet Dataset                      1GB  2019-01-29 22:45:54           4097        121  0.75             
devdgohil/the-oxfordiiit-pet-dataset                        The Oxford-IIIT Pet Dataset                    780MB  2019-06-14 20:03:38           1641         30  0.875            
chinmaychahar/iiit-word-level-handwritten-kannada-dataset   IIIT Word Level Handwritten Kannada Dataset      5GB  2023-02-25 07:06:16              0          0  0.125            
imbikramsaha/cat-breeds                                     Oxford IIIT Cats                             

In [12]:
!kaggle datasets download chinmaychahar/iiit-word-level-handwritten-kannada-dataset -p /content/sample_data/ --unzip

100% 4.82G/4.83G [00:42<00:00, 168MB/s]
100% 4.83G/4.83G [00:42<00:00, 121MB/s]


In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.7 MB/s eta 0:00:00


In [14]:
!pip install datasets jiwer evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [15]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from PIL import Image
from transformers import ViTFeatureExtractor, RobertaTokenizer, TrOCRProcessor
from transformers import VisionEncoderDecoderModel
from transformers import TrOCRProcessor
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import default_data_collator
from datasets import load_metric
os.environ["WANDB_DISABLED"] = "true"
# torch.cuda.empty_cache()

In [16]:
# directory and file paths
train_text_file = "/content/sample_data/kn/train.txt"
test_text_file = "/content/sample_data/kn/test.txt"
val_text_file = "/content/sample_data/kn/val.txt"
root_dir = "/content/sample_data/kn/"

In [18]:
def dataset_generator(data_path, set_name):
    with open(data_path) as f:
        dataset = f.readlines()
    counter = 0

    with open("/content/sample_data/kn/vocab.txt") as f:
        vocab = f.readlines()

    for j in range(len(vocab)):
        vocab[j] = vocab[j].split("\n")[0].strip()

    dataset_list = []
    for i in range(len(dataset)):
        if counter > 30000:
            break
        image_id = dataset[i].split("\n")[0].split(',')[0].strip()
        image_id = set_name + "/" + image_id
        vocab_id = int(dataset[i].split("\n")[0].split(',')[1].strip())
        text = vocab[vocab_id]
        row = [image_id, text]
        dataset_list.append(row)
        counter += 1

    dataset_df = pd.DataFrame(dataset_list, columns=['file_name', 'text'])
    return dataset_df
    
train_df = dataset_generator(train_text_file, 'train')
test_df = dataset_generator(test_text_file, 'test')
val_df = dataset_generator(val_text_file, 'val')

print(f"Train, Test & Val shape: {train_df.shape, test_df.shape, val_df.shape}")

Train, Test & Val shape: ((30001, 2), (15730, 2), (13752, 2))


In [19]:
for i in range(len(train_df)):
    if train_df['text'][i] == "" or train_df['text'][i] == " ":
        print(i)

In [20]:
for i in range(len(val_df)):
    if val_df['text'][i] == "" or val_df['text'][i] == " ":
        print(i)

In [21]:
for i in range(len(test_df)):
    if test_df['text'][i] == "" or test_df['text'][i] == " ":
        print(i)

In [22]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        # print(encoding)
        return encoding

In [23]:
encode = 'google/vit-base-patch16-224-in21k'
decode = 'Chakita/KannadaBERT'

feature_extractor=ViTFeatureExtractor.from_pretrained(encode)
tokenizer = RobertaTokenizer.from_pretrained(decode)
processor = TrOCRProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)

train_dataset = IAMDataset(root_dir=root_dir,
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir=root_dir,
                           df=test_df,
                           processor=processor)

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encode, decode)

model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
print(f"processor.tokenizer.pad_token_id: {processor.tokenizer.pad_token_id}")
model.config.vocab_size = model.config.decoder.vocab_size
# config_decoder.is_decoder = True
# config_decoder.add_cross_attention = True

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

/usr/local/lib/python3.8/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/transformers/models/trocr/processing_trocr.py:44: FutureWarning: The `feature_extractor` argument is deprecated and will be removed in v5, use `image_processor` instead.
  warnings.warn(


Some weights of RobertaForCausalLM were not initialized from the model checkpoint at Chakita/KannadaBERT and are newly initialized: ['roberta.encoder.layer.4.crossattention.self.key.bias', 'roberta.encoder.layer.5.crossattention.output.dense.weight', 'roberta.encoder.layer.0.crossattention.output.dense.bias', 'roberta.encoder.layer.1.crossattention.self.key.weight', 'roberta.encoder.layer.5.crossattention.output.dense.bias', 'roberta.encoder.layer.3.crossattention.output.dense.weight', 'roberta.encoder.layer.4.crossattention.self.query.bias', 'roberta.encoder.layer.5.crossattention.self.value.weight', 'roberta.encoder.layer.2.crossattention.self.query.bias', 'roberta.encoder.layer.5.crossattention.output.LayerNorm.weight', 'roberta.encoder.layer.2.crossattention.self.key.bias', 'roberta.encoder.layer.0.crossattention.self.query.bias', 'roberta.encoder.layer.0.crossattention.self.key.weight', 'roberta.encoder.layer.3.crossattention.self.query.bias', 'roberta.encoder.layer.3.crossattenti

processor.tokenizer.pad_token_id: 1


In [24]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 30001
Number of validation examples: 15730


In [25]:
training_args = Seq2SeqTrainingArguments(
    num_train_epochs=1,
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=4,
    output_dir="./checkpoints/",
    logging_steps=2,
    save_steps=2000,
    eval_steps=100,
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [26]:
import evaluate
cer_metric = evaluate.load("cer")

In [27]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return {"cer": cer}

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/models/trocr/processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 30001
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 15001
  Number of trainable parameters = 220989088


Step,Training Loss,Validation Loss
